In [1]:
REDOWNLOAD = False

In [2]:
import requests
import json
TFMRK_URL = "http://localhost:8000"

competition_ids = [
	"IT1",
	"GB1",
	"ES1",
	"FR1",
	"L1"
]

competition_clubs = {}
if REDOWNLOAD:
	for id in competition_ids:
		clubs = requests.get(TFMRK_URL+f"/competitions/{id}/clubs").content.decode()
		clubs = json.loads(clubs)
		competition_clubs[clubs['name']] = clubs['clubs']
	
	with open('clubs.json', 'w') as f:
		json.dump(competition_clubs, f, indent=4)
else:
	with open('clubs.json', 'r') as f:
		competition_clubs = json.load(f)

In [3]:
club_players = {}

if REDOWNLOAD:
	for key, value in competition_clubs.items():
		for club in value:
			players = requests.get(TFMRK_URL+f"/clubs/{club['id']}/players").content.decode()
			players = json.loads(players)
			club_players[club['name']] = players['players']

	with open('players.json', 'w') as f:
		json.dump(club_players, f, indent=4)
else:
	with open('players.json', 'r') as f:
		club_players = json.load(f)

In [4]:
club_players

{'Inter Milan': [{'id': '256339',
   'name': 'Emil Audero',
   'position': 'Goalkeeper',
   'dateOfBirth': 'Jan 18, 1997',
   'age': '27',
   'nationality': ['Italy'],
   'height': '1,92m',
   'foot': 'right',
   'joinedOn': 'Aug 10, 2023',
   'joined': 'On loan from UC Sampdoria until Jun 30, 2024',
   'signedFrom': 'UC Sampdoria',
   'contract': '€6.00m',
   'marketValue': '€6.00m'},
  {'id': '42205',
   'name': 'Yann Sommer',
   'position': 'Goalkeeper',
   'dateOfBirth': 'Dec 17, 1988',
   'age': '35',
   'nationality': ['Switzerland'],
   'height': '1,83m',
   'foot': 'right',
   'joinedOn': 'Aug 7, 2023',
   'joined': 'Joined from Bayern Munich; date: Aug 7, 2023; fee: €6.75m',
   'signedFrom': 'Bayern Munich',
   'contract': '€5.00m',
   'marketValue': '€5.00m'},
  {'id': '70580',
   'name': 'Raffaele Di Gennaro',
   'position': 'Goalkeeper',
   'dateOfBirth': 'Oct 3, 1993',
   'age': '30',
   'nationality': ['Italy'],
   'height': '1,88m',
   'foot': 'right',
   'joinedOn': 'Ju

In [5]:
player_evaluations = {}
count = 0
if REDOWNLOAD:
	for key, value in club_players.items():
		print("---------",key, "---------")
		for player in value:
			evaluation = requests.get(TFMRK_URL+f"/players/{player['id']}/market_value").content.decode()
			evaluation = json.loads(evaluation)
			player_evaluations[player['name']] = evaluation['marketValueHistory']
			print(player['name'])
			if count == 50:
				break
			count += 1
		break

	with open('evaluations.json', 'w') as f:
		json.dump(player_evaluations, f, indent=4)
else:
	with open('evaluations.json', 'r') as f:
		player_evaluations = json.load(f)

In [6]:
player_evaluations

{'Emil Audero': [{'age': '16',
   'date': 'Oct 25, 2013',
   'clubName': 'Juventus Primavera',
   'value': 100000.0,
   'clubID': '506'},
  {'age': '17',
   'date': 'Jun 24, 2014',
   'clubName': 'Juventus Primavera',
   'value': 100000.0,
   'clubID': '506'},
  {'age': '18',
   'date': 'Jun 17, 2015',
   'clubName': 'Juventus Primavera',
   'value': 175000.0,
   'clubID': '506'},
  {'age': '18',
   'date': 'Jan 4, 2016',
   'clubName': 'Juventus FC',
   'value': 200000.0,
   'clubID': '506'},
  {'age': '19',
   'date': 'Jul 15, 2016',
   'clubName': 'Juventus FC',
   'value': 300000.0,
   'clubID': '506'},
  {'age': '19',
   'date': 'Jan 2, 2017',
   'clubName': 'Juventus FC',
   'value': 300000.0,
   'clubID': '506'},
  {'age': '20',
   'date': 'Jun 8, 2017',
   'clubName': 'Juventus FC',
   'value': 500000.0,
   'clubID': '506'},
  {'age': '20',
   'date': 'Dec 21, 2017',
   'clubName': 'Venezia FC',
   'value': 1300000.0,
   'clubID': '607'},
  {'age': '21',
   'date': 'Jun 18, 201

In [7]:
def numerize(s):
	multipliers = {'k': 10**3, 'm': 10**6, 'b': 10**9}
	number = float(s[:-1])
	if s[-1] in multipliers:
		return number * multipliers[s[-1]]
	else:
		return number

if REDOWNLOAD:    
	for key, value in player_evaluations.items():
		print("---------",key, "---------")
		for evaluation in value:
			# print(evaluation['date'], numerize(evaluation['value'][1:]))
			evaluation['value'] = numerize(evaluation['value'][1:])
			player_evaluations[key] = value

	with open('evaluations.json', 'w') as f:
		json.dump(player_evaluations, f, indent=4)
else:
	with open('evaluations.json', 'r') as f:
		player_evaluations = json.load(f)

In [8]:
import pandas as pd

df = pd.json_normalize(player_evaluations)
df = df.T
df

,0
Emil Audero,"[{'age': '16', 'date': 'Oct 25, 2013', 'clubNa..."
Yann Sommer,"[{'age': '17', 'date': 'Jul 22, 2006', 'clubNa..."
Raffaele Di Gennaro,"[{'age': '17', 'date': 'Nov 16, 2010', 'clubNa..."
Alessandro Bastoni,"[{'age': '17', 'date': 'Oct 17, 2016', 'clubNa..."
Benjamin Pavard,"[{'age': '19', 'date': 'Jul 2, 2015', 'clubNam..."
Yann Bisseck,"[{'age': '17', 'date': 'Dec 28, 2017', 'clubNa..."
Stefan de Vrij,"[{'age': '17', 'date': 'Oct 2, 2009', 'clubNam..."
Francesco Acerbi,"[{'age': '22', 'date': 'Oct 8, 2010', 'clubNam..."
Federico Dimarco,"[{'age': '16', 'date': 'Sep 16, 2014', 'clubNa..."
Matteo Darmian,"[{'age': '17', 'date': 'Aug 3, 2007', 'clubNam..."


In [29]:
def create_row(row):
    minidf = pd.json_normalize(row[0])
    minidf['year'] = minidf['date'].apply(lambda x: pd.to_datetime(x).year)
    max_value = minidf.groupby('year')['value'].max()
    for evaluation in minidf.iterrows():
        row[evaluation[1]['year']] = max_value.loc[evaluation[1]['year']]
    return row

players_evaluations_for_year = df.apply(create_row, axis=1)

In [ ]:
players_evaluations_for_year

In [4]:
years = [
    (2018, 2019),
    (2019, 2020),
    (2020, 2021),
    (2021, 2022),
    (2022, 2023),
]
leagues = [
    "Serie-A",
    "Premier-League",
    "La-Liga",
    "Bundesliga",
    "Ligue-1"
]

In [7]:
import os
from tqdm import tqdm
import swifter
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask.distributed import Client, progress
import dask
import pandas as pd
tqdm.pandas()
client = Client(threads_per_worker=2, n_workers=10)

def numerize(s):
	multipliers = {'k': 10**3, 'm': 10**6, 'b': 10**9}
	number = float(s[:-1])
	if s[-1] in multipliers:
		return number * multipliers[s[-1]]
	else:
		return number

def find_evaluation(row, year):
    try:
        print(row['Player'] + " enter" + str(year))
        find = f"{TFMRK_URL}/players/search/{row['Player']}?page_number=1"
        result = requests.get(find).content.decode()
        result = json.loads(result)
        if len(result['results']) == 0:
            row["value"] = None
        player_id = result['results'][0]['id']
        evaluation = requests.get(TFMRK_URL+f"/players/{player_id}/market_value").content.decode()
        mini_df = pd.json_normalize(json.loads(evaluation)['marketValueHistory'])
        mini_df['year'] = mini_df['date'].apply(lambda x: pd.to_datetime(x).year)
        max_value = mini_df.groupby('year')['value'].max()
        max_value = numerize(max_value.loc[year[1]][1:])
        row["value"] = max_value
    except Exception as e:
        row["value"] = None
    print(row['Player'] + " exit" + str(year))
    # pbar.update(1)
    return row


def sqad_with_data(squad, year, s, league):
    print(s)
    ddf = dd.from_pandas(squad, npartitions=10)
    with ProgressBar():
        squad = ddf.apply(find_evaluation, axis=1, year=year, meta=(None, 'object')).compute()
        squad.to_csv(f"./data/whole/{s}-{league}-with_value.csv", index=False)
        print(f"./data/whole/{s}-{league}-with_value.csv end")


lazy = []
for league in leagues:
    for year in years:
        for s in os.listdir(f"./data/{league}/{year}"):
            if "whole" in s:
                squad = pd.read_csv(f"./data/{league}/{year}/{s}")
                res = dask.delayed(sqad_with_data)(squad, year, s, league)
                lazy.append(res)

/home/matteo/miniconda3/envs/top5/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35103 instead
  warnings.warn(


In [8]:
dask.compute(*lazy)

whole-2019,2020.csvwhole-2022,2023.csv

whole-2022,2023.csv
whole-2018,2019.csv
whole-2018,2019.csv
whole-2020,2021.csvwhole-2020,2021.csv

whole-2018,2019.csvwhole-2020,2021.csv

whole-2020,2021.csv
whole-2021,2022.csv
whole-2022,2023.csv
whole-2018,2019.csv
whole-2019,2020.csv
whole-2022,2023.csv
whole-2019,2020.csvwhole-2021,2022.csv
whole-2019,2020.csv
whole-2022,2023.csv
whole-2019,2020.csv

whole-2018,2019.csv
whole-2020,2021.csv
whole-2021,2022.csv
whole-2021,2022.csv
whole-2021,2022.csv
Patrick Burner enter(2020, 2021)
Toni Villa enter(2018, 2019)
Franco Di Santo enter(2018, 2019)
José Campaña enter(2018, 2019)
Guilherme Arana enter(2018, 2019)
Romain Faivre enter(2020, 2021)
Hodei Oleaga enter(2018, 2019)
Marcus Coco enter(2020, 2021)
Anaitz Arbilla enter(2018, 2019)
Xavier Chavalerin enter(2020, 2021)Francisco Portillo enter(2018, 2019)

Miguel Trauco enter(2020, 2021)
Terem Moffi enter(2020, 2021)
Dimitri Payet enter(2020, 2021)
Ruben Aguilar enter(2020, 2021)
Thomas Vermael

/tmp/ipykernel_8891/3686785082.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_8891/3686785082.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Marshall Munetsi exit(2020, 2021)
Valon Berisha enter(2020, 2021)
Daniele Verde exit(2018, 2019)
Leonardo Suárez enter(2018, 2019)
Wahbi Khazri exit(2020, 2021)
Saidou Sow enter(2020, 2021)
Jean-Kevin Duverne exit(2020, 2021)
Irvin Cardona enter(2020, 2021)
Carlos Akapo exit(2018, 2019)
Aleksandar Jovanović enter(2018, 2019)
Naomichi Ueda exit(2020, 2021)
Nolan Roux enter(2020, 2021)
Martín Pascual exit(2018, 2019)
Manuel Navarro enter(2018, 2019)
Houboulang Mendes exit(2020, 2021)
Matthieu Dreyer enter(2020, 2021)
Juan Brandáriz exit(2018, 2019)
Moussa Wagué enter(2018, 2019)
Valon Berisha exit(2020, 2021)
Arber Zeneli enter(2020, 2021)
Bridge Ndilu exit(2020, 2021)Nolito exit(2018, 2019)

Batista Mendy enter(2020, 2021)Marko Rog enter(2018, 2019)

Tim Oermann exit(2022, 2023)
Jacek Góralski enter(2022, 2023)
Ximo Navarro exit(2018, 2019)
Manu García enter(2018, 2019)
Kiké exit(2018, 2019)
Sergio Álvarez enter(2018, 2019)
Sebastián Cristóforo exit(2018, 2019)Saidou Sow exit(2020, 2021

/tmp/ipykernel_8891/4058130717.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_8891/4058130717.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Albin Ekdal exit(2018, 2019)
Karol Linetty enter(2018, 2019)
Péter Gulácsi exit(2018, 2019)
Ibrahima Konaté enter(2018, 2019)
Virgile Pinson exit(2018, 2019)
Grejohn Kyei enter(2018, 2019)
Noah König exit(2021, 2022)
Jonas Hector enter(2021, 2022)
Samuel Koeberlé exit(2021, 2022)
Enzo Valentim enter(2021, 2022)
Willy Caballero exit(2021, 2022)
William Smallbone enter(2021, 2022)
Issa Diop exit(2019, 2020)
Angelo Ogbonna enter(2019, 2020)
Diego Carlos exit(2021, 2022)
Jules Koundé enter(2021, 2022)
Giorgos Kyriakopoulos exit(2021, 2022)Jack Robinson exit(2020, 2021)
Billy Sharp enter(2020, 2021)

Hamed Junior Traorè enter(2021, 2022)
Yussuf Poulsen exit(2021, 2022)
Marcel Halstenberg enter(2021, 2022)
Joël Matip exit(2022, 2023)
Diogo Jota enter(2022, 2023)
Karol Linetty exit(2018, 2019)
Grégoire Defrel enter(2018, 2019)
James Ward-Prowse exit(2022, 2023)
Gavin Bazunu enter(2022, 2023)
Virgil Misidjan exit(2018, 2019)
Ewerton enter(2018, 2019)
El Bilal Touré exit(2022, 2023)
Nathanael M

...

/tmp/ipykernel_8891/4058130717.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_8891/4058130717.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Raúl García exit(2020, 2021)
Mikel Vesga enter(2020, 2021)
Jeffrey Gouweleeuw exit(2020, 2021)
Daniel Caligiuri enter(2020, 2021)
Nicola Sansone exit(2022, 2023)
Joshua Zirkzee enter(2022, 2023)
Mikel Vesga exit(2020, 2021)
Óscar de Marcos enter(2020, 2021)
Nenad Tomović exit(2018, 2019)
Filip Đorđević enter(2018, 2019)
Myles Peart-Harris exit(2021, 2022)
Nathan Young-Coombes enter(2021, 2022)
Filip Đorđević exit(2018, 2019)
Fabrizio Cacciatore enter(2018, 2019)
Daniel Caligiuri exit(2020, 2021)
Ohis Felix Uduokhai enter(2020, 2021)
Joshua Zirkzee exit(2022, 2023)
Lorenzo De Silvestri enter(2022, 2023)
Fabrizio Cacciatore exit(2018, 2019)
Joel Obi enter(2018, 2019)
Nathan Young-Coombes exit(2021, 2022)
Daniel Oyegoke enter(2021, 2022)
Iru exit(2021, 2022)
./data/whole/whole-2021,2022.csv-La-Liga-with_value.csv end
Óscar de Marcos exit(2020, 2021)
Yuri Berchiche enter(2020, 2021)


/tmp/ipykernel_8891/4058130717.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_8891/4058130717.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Joel Obi exit(2018, 2019)
Emanuel Vignato enter(2018, 2019)
Ohis Felix Uduokhai exit(2020, 2021)
Carlos Gruezo enter(2020, 2021)
Daniel Oyegoke exit(2021, 2022)
Patrik Gunnarsson enter(2021, 2022)
Lorenzo De Silvestri exit(2022, 2023)
Joaquin Sosa enter(2022, 2023)
Yuri Berchiche exit(2020, 2021)
Mikel Balenziaga enter(2020, 2021)
Emanuel Vignato exit(2018, 2019)
Marco Andreolli enter(2018, 2019)
Carlos Gruezo exit(2020, 2021)
André Hahn enter(2020, 2021)
Joaquin Sosa exit(2022, 2023)
Giorgos Kyriakopoulos enter(2022, 2023)
Patrik Gunnarsson exit(2021, 2022)
Matthew Cox enter(2021, 2022)
Mikel Balenziaga exit(2020, 2021)
Unai Vencedor Paris enter(2020, 2021)
André Hahn exit(2020, 2021)
Rani Khedira enter(2020, 2021)
Matthew Cox exit(2021, 2022)
Maxwell Haygarth enter(2021, 2022)
Giorgos Kyriakopoulos exit(2022, 2023)
Denso Kasius enter(2022, 2023)
Maxwell Haygarth exit(2021, 2022)
Edmond-Paris Maghoma enter(2021, 2022)
Unai Vencedor Paris exit(2020, 2021)
Asier Villalibre enter(2020, 2

(None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None)

In [63]:
len(os.sched_getaffinity(0))

20

In [37]:
squad

,Player,Nation,Pos,Age,MP,Playing Time_Starts,Playing Time.1_Min,Playing Time.2_90s,Performance_Gls,Performance.1_Ast,...,Per 90 Minutes_Gls,Per 90 Minutes.1_Ast,Per 90 Minutes.2_G+A,Per 90 Minutes.3_G-PK,Per 90 Minutes.4_G+A-PK,Per 90 Minutes.5_xG,Per 90 Minutes.6_xAG,Per 90 Minutes.7_xG+xAG,Per 90 Minutes.8_npxG,Per 90 Minutes.9_npxG+xAG
0,Lautaro Martínez,ar ARG,FW,24,57,45,4112.0,45.7,28.0,10.0,...,0.61,0.22,0.83,0.59,0.81,0.53,0.18,0.71,0.49,0.67
1,Nicolò Barella,it ITA,MF,25,52,48,4011.0,44.6,9.0,9.0,...,0.20,0.20,0.40,0.20,0.40,0.12,0.16,0.27,0.12,0.27
2,Francesco Acerbi,it ITA,DF,34,49,40,3836.0,42.6,1.0,2.0,...,0.02,0.05,0.07,0.02,0.07,0.04,0.03,0.07,0.04,0.07
3,André Onana,cm CMR,GK,26,41,41,3720.0,41.3,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Alessandro Bastoni,it ITA,DF,23,46,43,3525.0,39.2,0.0,6.0,...,0.00,0.15,0.15,0.00,0.15,0.04,0.13,0.17,0.04,0.17
5,Hakan Çalhanoğlu,tr TUR,MF,28,49,41,3484.0,38.7,4.0,8.0,...,0.10,0.21,0.31,0.08,0.28,0.10,0.18,0.28,0.08,0.26
6,Denzel Dumfries,nl NED,DF,26,51,39,3435.0,38.2,2.0,4.0,...,0.05,0.10,0.16,0.05,0.16,0.16,0.15,0.31,0.16,0.31
7,Matteo Darmian,it ITA,DF,32,48,38,3336.0,37.1,2.0,2.0,...,0.05,0.05,0.11,0.05,0.11,0.03,0.07,0.10,0.03,0.10
8,Henrikh Mkhitaryan,am ARM,MF,33,49,40,3318.0,36.9,5.0,2.0,...,0.14,0.05,0.19,0.14,0.19,0.14,0.08,0.22,0.14,0.22
9,Federico Dimarco,it ITA,DF,24,50,40,3146.0,35.0,6.0,8.0,...,0.17,0.23,0.40,0.17,0.40,0.14,0.32,0.46,0.14,0.46


In [43]:
def get_player_value(row):
    try:
        evaluation = players_evaluations_for_year[players_evaluations_for_year.index == row['Player']]
        row['value'] = evaluation[2023][0]
    except:
        row['value'] = "NA"
    return row

squad.apply(get_player_value, axis=1).to_csv("test.csv")

/tmp/ipykernel_845/2557236367.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row['value'] = evaluation[2023][0]
/tmp/ipykernel_845/2557236367.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row['value'] = evaluation[2023][0]
/tmp/ipykernel_845/2557236367.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row['value'] = evaluation[2023][0]
/tmp/ipykernel_845/2557236367.py:4: FutureWarning: Series.__getitem__ treating keys a